# Random Forest vs AdaBoost: Comparação e Otimização
---

## **1. Diferenças Fundamentais entre Random Forest e AdaBoost**

O **Random Forest** e o **AdaBoost** são dois algoritmos poderosos da família de **ensemble learning**, mas possuem diferenças fundamentais em sua concepção e funcionamento. A seguir, destacamos **cinco principais diferenças** entre esses dois métodos:

### **Estratégia de Construção do Modelo**
- **Random Forest (RF)**: Constrói **múltiplas árvores de decisão independentes** em paralelo, cada uma treinada em uma amostra bootstrap do conjunto de dados. A previsão final é obtida por **votação majoritária** (classificação) ou **média das previsões** (regressão).
  
  $$ \hat{y}_{RF} = \frac{1}{T} \sum_{t=1}^{T} f_t(x) $$

- **AdaBoost (Adaptive Boosting)**: Constrói árvores sequencialmente, onde cada modelo tenta **corrigir os erros do modelo anterior**. Os exemplos mal classificados recebem pesos maiores para influenciar mais a próxima árvore.

### **Redução do Erro**
- **Random Forest**: Reduz a **variância** ao combinar várias árvores de decisão, melhorando a **generalização do modelo**.
- **AdaBoost**: Foca na **redução do viés**, aprimorando a **capacidade do modelo de aprender padrões complexos**, mas pode ser mais sensível a **outliers**.

### **Complexidade dos Modelos Base**
- **Random Forest**: Usa **árvores profundas** como base learners.
- **AdaBoost**: Frequentemente utiliza **árvores rasas** (stumps, com `max_depth=1`) como base learners.

### **Ponderação das Previsões**
- **Random Forest**: Todas as árvores têm **peso igual** no ensemble.
- **AdaBoost**: Árvores com **melhor desempenho** recebem maior peso, enquanto modelos fracos têm menos influência.

  $$ \alpha_t = \frac{1}{2} \ln \left( \frac{1 - e_t}{e_t} \right) $$

  onde:
  - $e_t$ é o erro da $t$-ésima árvore.
  - $\alpha_t$ é o peso atribuído à árvore na combinação final.

### **Sensibilidade a Dados Ruidosos**
- **Random Forest**: Mais **robusto a outliers** devido ao uso do **bootstrap** e da agregação de múltiplos modelos.
- **AdaBoost**: Pode ser **mais sensível** a outliers, pois foca mais nos exemplos difíceis de classificar.

---

## **2. Implementação do AdaBoost no Scikit-Learn**
O código abaixo implementa o **AdaBoostClassifier** utilizando o conjunto de dados **Iris**. O desempenho do modelo é avaliado por **validação cruzada** (`cross_val_score`).

In [1]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets        import load_iris
from sklearn.ensemble        import AdaBoostClassifier

In [2]:
X, y   = load_iris(return_X_y=True)

clf    = AdaBoostClassifier(n_estimators=100)

scores = cross_val_score(estimator=clf, 
                         X=X, 
                         y=y, 
                         cv=5)

scores.mean()

0.9466666666666665

## **3. Principais Hiperparâmetros do AdaBoost**
O desempenho do AdaBoost pode ser otimizado ajustando seus **hiperparâmetros**. Aqui estão os mais importantes:

#### **`n_estimators` (Número de Estimadores)**
- Define o **número de aprendizes fracos**.
- Maior valor tende a melhorar a acurácia, mas pode causar **overfitting**.

#### **`learning_rate` (Taxa de Aprendizado)**
- Controla a contribuição de cada classificador no ensemble.
- Valores muito altos podem causar **oscilação**, enquanto valores muito baixos podem dificultar a convergência.

#### **`base_estimator` (Modelo Base)**
- Define o **aprendiz fraco** usado no boosting (padrão: `DecisionTreeClassifier(max_depth=1)`).

#### **`algorithm` (Algoritmo de Atualização)**
- `SAMME`: Versão discreta do AdaBoost (usa pesos de classificação).
- `SAMME.R`: Versão que usa probabilidades e geralmente tem **melhor desempenho**.

#### **`random_state` (Reprodutibilidade)**
- Define um **seed** para manter resultados consistentes.

---

## **4. Otimização do AdaBoost com GridSearch**
Ajustamos o hiperparâmetro **`n_estimators`** para encontrar o número ideal de estimadores.

In [3]:
import pandas as pd

In [4]:
%%time

estimators = list(range(1, 1002, 100))

n_estimators = []
mean_scores  = []

for n in estimators:
    clf    = AdaBoostClassifier(n_estimators=n)
    scores = cross_val_score(estimator=clf, 
                             X=X, 
                             y=y, 
                             cv=5)
    n_estimators.append(n)
    mean_scores.append(scores.mean())

pd.DataFrame(data=list(zip(n_estimators, mean_scores)), 
             columns=['n_estimators', 'mean_score'])

CPU times: user 45.4 s, sys: 582 ms, total: 45.9 s
Wall time: 46.8 s


,n_estimators,mean_score
0,1,0.666667
1,101,0.953333
2,201,0.946667
3,301,0.946667
4,401,0.946667
5,501,0.946667
6,601,0.946667
7,701,0.946667
8,801,0.946667
9,901,0.946667


---

### **Interpretação dos Resultados**
- **Com `n_estimators=1`**, a acurácia é baixa (66.67%), indicando um modelo muito fraco.
- A acurácia **aumenta rapidamente** até `n_estimators=101`, atingindo 95.33%.
- Após **201 estimadores**, a melhoria é marginal, sugerindo que adicionar mais árvores **não traz ganhos significativos**.

Um bom ponto de equilíbrio entre desempenho e eficiência computacional pode ser **`n_estimators=101`**.

---

## **Conclusão**
- **Random Forest** é mais adequado para **redução de variância**, sendo mais robusto a outliers e overfitting.  
- **AdaBoost** foca na **redução do viés**, priorizando exemplos mal classificados para melhorar a aprendizagem.  
- **Otimizar hiperparâmetros**, como `n_estimators` e `learning_rate`, melhora significativamente o desempenho do AdaBoost.  

A escolha entre **Random Forest** e **AdaBoost** deve considerar **as características do problema** e **a necessidade de equilíbrio entre viés e variância**. 